In [37]:
# Init environment
import docker
import python_on_whales as pow

client = docker.from_env()
pow_docker = pow.docker


def printAllImages():
    imgs = client.images.list()
    for img in imgs:
        print(img)

TASK 2

In [38]:
# Define the images needed to use in task 2
task2Imgs = ['locustio/locust','mongo','mongo-express','gcr.io/cadvisor/cadvisor','dockersamples/visualizer']

# Show all the downloaded images.
printAllImages()


<Image: 'chenbao0707/cadvisor-scrapper:latest', 'cadvisor-scraper:latest'>
<Image: ''>
<Image: 'alpine/git:latest'>
<Image: 'mongo:latest'>
<Image: 'locustio/locust:latest'>
<Image: 'mongo-express:latest'>
<Image: 'ubuntu:latest'>
<Image: 'dockersamples/visualizer:latest'>
<Image: 'gcr.io/cadvisor/cadvisor:latest'>
<Image: 'nclcloudcomputing/javabenchmarkapp:latest'>


In [34]:
"""
# 1. delete all the images that having downloaded by CLI.

imgs = client.images.list()
for task2Img in task2Imgs:
    try:
        client.images.remove(image = task2Img, force = True)
        print("Image [", task2Img , "] has been deleted successfully.")
    except BaseException:
        print("No such image [", task2Img , "], no need to delete it.")

# Check if those images have been deleted successfully.
printAllImages()

"""


No such image [ locustio/locust ], need not to delete it.
No such image [ mongo ], need not to delete it.
No such image [ mongo-express ], need not to delete it.
No such image [ gcr.io/cadvisor/cadvisor ], need not to delete it.
No such image [ dockersamples/visualizer ], need not to delete it.
<Image: 'chenbao0707/cadvisor-scrapper:latest', 'cadvisor-scraper:latest'>
<Image: 'alpine/git:latest'>
<Image: 'ubuntu:latest'>
<Image: 'nclcloudcomputing/javabenchmarkapp:latest'>


In [40]:
# If those images do not exist, download them from Docker Hub.
imgs = client.images.list()
for task2Img in task2Imgs:
    exist = False
    for img in imgs:
        if  len(img.tags) > 0 and img.tags[0] == task2Img + ':latest':
            exist = True
    print("Image [{}] exists or not: {}".format(task2Img, exist))
    if not exist:
        # download if not eexist
        print("Image [{}] not exist, download it now.".format(task2Img))
        client.images.pull(repository = task2Img)

Image [locustio/locust] exists or not: True
Image [mongo] exists or not: True
Image [mongo-express] exists or not: True
Image [gcr.io/cadvisor/cadvisor] exists or not: True
Image [dockersamples/visualizer] exists or not: True


In [15]:
# Check if those images have been deleted successfully.
printAllImages()

<Image: 'chenbao0707/cadvisor-scrapper:latest', 'cadvisor-scraper:latest'>
<Image: 'alpine/git:latest'>
<Image: 'mongo:latest'>
<Image: 'locustio/locust:latest'>
<Image: 'mongo-express:latest'>
<Image: 'ubuntu:latest'>
<Image: 'dockersamples/visualizer:latest'>
<Image: 'gcr.io/cadvisor/cadvisor:latest'>
<Image: 'nclcloudcomputing/javabenchmarkapp:latest'>


In [51]:
# Anyway, remove those services firstly.
pow_docker.stack.remove('nclbenchmark')


In [54]:
# Start up the compose services in swarm
pow_docker.stack.deploy(name='nclbenchmark', compose_files= 'task2.yaml')


In [55]:
# Show if those services have bean started up successfully
client.services.list()

[<Service: g4th8mwze5>,
 <Service: nyn7ovesv8>,
 <Service: sjzazsyrmw>,
 <Service: wc3wdvx8uk>,
 <Service: xp63zkm9af>,
 <Service: ylllwbqbkk>]